In [1]:
!pwd

/home/rokey/git_package/cobot2/unit-tasks/Object_Detection


In [3]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os

# 정확한 상대 경로 지정
save_path = "."
os.makedirs(save_path, exist_ok=True)  # 폴더 없을 때만 생성
filename = os.path.join(save_path, "capture6.png")
# filename = os.path.join(save_path, "capture2.png")

# RealSense 파이프라인 설정
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# 카메라 실행 및 한 프레임 저장
pipeline.start(config)
try:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    if color_frame:
        color_image = np.asanyarray(color_frame.get_data())
        cv2.imwrite(filename, color_image)
        print(f"캡처 완료: {filename}")
    else:
        print("프레임을 가져올 수 없습니다.")
finally:
    pipeline.stop()


캡처 완료: ./capture6.png


In [39]:
## 캡처할 때 부터 이미지 개선
import pyrealsense2 as rs
import numpy as np
import cv2
import os

# 저장 경로
save_path = "."
os.makedirs(save_path, exist_ok=True)
filename = os.path.join(save_path, "capture5.png")

# RealSense 파이프라인 설정
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)  # 더 높은 해상도

# 파이프라인 시작
profile = pipeline.start(config)
device = profile.get_device()
color_sensor = device.query_sensors()[1]  # 보통 컬러 센서는 index 1


#수동 튜닝
color_sensor.set_option(rs.option.enable_auto_exposure, 0)  # 자동 노출 끄기

color_sensor.set_option(rs.option.exposure, 70)           # 이전보다 약간 올려서 더 밝게
color_sensor.set_option(rs.option.brightness, 25)         # 미세 조정
color_sensor.set_option(rs.option.contrast, 60)           # ✅ contrast 줄임
color_sensor.set_option(rs.option.gamma, 200)             # ✅ gamma 줄임
color_sensor.set_option(rs.option.saturation, 60)         # 색 너무 세지 않도록
color_sensor.set_option(rs.option.white_balance, 4700)    # 약간 따뜻하게


# 안정적인 프레임 확보 후 캡처
try:
    for _ in range(10):  # warm-up
        frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    if color_frame:
        color_image = np.asanyarray(color_frame.get_data())
        cv2.imwrite(filename, color_image)
        print(f"✅ 캡처 완료: {filename}")
    else:
        print("❌ 컬러 프레임을 가져오지 못했습니다.")
finally:
    pipeline.stop()

✅ 캡처 완료: ./capture5.png


In [ ]:
## 참고 카메라 밝기 확인
def print_sensor_range(sensor):
    for opt in [rs.option.brightness, rs.option.contrast, rs.option.saturation,
                rs.option.exposure, rs.option.white_balance, rs.option.sharpness, rs.option.gamma]:
        if sensor.supports(opt):
            r = sensor.get_option_range(opt)
            print(f"{opt.name:20} : min={r.min}, max={r.max}, default={r.default}, step={r.step}")

print_sensor_range(color_sensor)


brightness           : min=-64.0, max=64.0, default=0.0, step=1.0
contrast             : min=0.0, max=100.0, default=50.0, step=1.0
saturation           : min=0.0, max=100.0, default=64.0, step=1.0
exposure             : min=1.0, max=10000.0, default=166.0, step=1.0
white_balance        : min=2800.0, max=6500.0, default=4600.0, step=10.0
sharpness            : min=0.0, max=100.0, default=50.0, step=1.0
gamma                : min=100.0, max=500.0, default=300.0, step=1.0


In [14]:
## 전처리
import cv2
import numpy as np

def enhance_contrast_for_detection(image):
    # 감마 보정 (살짝 밝게)
    gamma = 1.3
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    gamma_corrected = cv2.LUT(image, table)

    # LAB 기반 CLAHE로 국소 대비 향상
    lab = cv2.cvtColor(gamma_corrected, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    final = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    return final


def sharpen_image(image):
    """
    윤곽을 강조하는 샤프닝 필터 적용
    """
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])  # 기본 sharpening kernel
    return cv2.filter2D(image, -1, kernel)

In [15]:
## 전처리 + 테스트

img = cv2.imread("capture.png")
# 전처리 적용
enhanced = enhance_contrast_for_detection(img)

# (선택) 샤프닝 추가
enhanced = sharpen_image(enhanced)

# YOLO 예측
from ultralytics import YOLO
model = YOLO("/home/kiwi/github_package/cobot2/unit-tasks/Object_Detection/yolo8_2.pt")
results = model.predict(enhanced, conf=0.25, save=True)


0: 480x640 2 hammers, 1 wrench, 281.1ms
Speed: 4.1ms preprocess, 281.1ms inference, 5.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict4


In [46]:
## 노랑배경 흰색으로
import cv2
import numpy as np

def replace_yellow_background_with_white(image_bgr):
    hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
    
    # 더 좁은 노란색 범위
    lower_yellow = np.array([22, 100, 100])
    upper_yellow = np.array([32, 255, 255])
    
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    result = image_bgr.copy()
    result[mask > 0] = [255, 255, 255]
    return result


img = cv2.imread("capture5.png")
processed_bgr = replace_yellow_background_with_white(img)

# 3. BGR → RGB 변환 (YOLO는 RGB 입력)
processed_rgb = cv2.cvtColor(processed_bgr, cv2.COLOR_BGR2RGB)

# 4. YOLO 모델 불러오기
model = YOLO("/home/kiwi/github_package/cobot2/unit-tasks/Object_Detection/yolo8_2.pt")

# 5. 예측 (save=False로 저장 생략)
results = model.predict(source=processed_rgb, save=True, save_txt=False, conf=0.25)


0: 384x640 2 hammers, 1 wrench, 237.8ms
Speed: 2.0ms preprocess, 237.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict18


In [5]:
## 이미지 테스트 yolo8
# /content/runs/detect/predict/  ← 결과 이미지 (.jpg) 및 .txt 파일
from ultralytics import YOLO

# 모델 로드
model = YOLO("/home/kiwi/github_package/cobot2/unit-tasks/Object_Detection/yolo8_2.pt")
# 예측 실행
results = model.predict(
    source="/home/kiwi/github_package/cobot2/unit-tasks/Object_Detection/capture6.png",
    save=True,      # 결과 이미지 저장
    save_txt=True,  # 라벨 텍스트 저장 (optional)
    conf=0.25        # confidence threshold (기본값은 0.25)
)


image 1/1 /home/kiwi/github_package/cobot2/unit-tasks/Object_Detection/capture6.png: 480x640 1 hammer, 1 wrench, 295.6ms
Speed: 4.7ms preprocess, 295.6ms inference, 9.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict20
1 label saved to runs/detect/predict20/labels


In [ ]:
## 이미지 테스트 yolo11
# /content/runs/detect/predict/  ← 결과 이미지 (.jpg) 및 .txt 파일
from ultralytics import YOLO

# 모델 로드
model = YOLO("/home/rokey/git_package/cobot2/unit-tasks/Object_Detection/yolo11_1.pt")

# 예측 실행
results = model.predict(
    source="/home/rokey/git_package/cobot2/unit-tasks/Object_Detection/capture6.png",
    save=True,      # 결과 이미지 저장
    save_txt=True,  # 라벨 텍스트 저장 (optional)
    conf=0.25        # confidence threshold (기본값은 0.25)
)


image 1/1 /home/rokey/git_package/cobot2/unit-tasks/Object_Detection/capture6.png: 480x640 1 hammer, 1 screwdriver, 1 wrench, 77.8ms
Speed: 1.0ms preprocess, 77.8ms inference, 64.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict19
1 label saved to runs/detect/predict19/labels
